# Linear regression

$D = (X, Y) = \{(x_i, y_i)\}_{i=1}^N$ - обучающая выборка, N - число объектов в выборке  
$X = \{x_{ij}\}$ - матрица признаков (design matrix), $X \in R^{Nxd}$,  
где d - размерность признаков.  
$Y = [y_1, ..., y_N]^T$ - целевой признак, значение которого необходимо научится предсказывать.  
В случае регрессии $Y \in R^N$


Определим ошибку на одном объекте выборки (loss): $Loss(y_i, \hat{y_i}) = (y_i - \hat{y_i})^2$    
Определим ожидаемую ошибку на всей выборке (в нашем случае это будет - MSE, Mean Square Error): $Q(f, x, y) = \frac{1}{N} \sum_{i=1}^N Loss(y_i, f(x_i))$  

где f(x) - наша модель регрессии  

Рассмотрим модель линейной регрессии: $f(x_i) = f(x_i; w) = \sum_{j=1}^d w_j x_{ij} + w_0 = W^TX + w_0$,  
где $W = [w_1, ..., w_d]^T$ - вектор весов, который нужно обучить.  

Тогда ожидаемую ошибку на всей выборке можно записать как:  
$Q(f, x, y) = \frac{1}{N} \sum_{i=1}^N Loss(y_i, f(x_i)) = \frac{1}{N} \sum_{i=1}^N Loss(y_i, \sum_{j=1}^d w_j x_{ij} + w_0) = \frac{1}{N} \sum_{i=1}^N (y_i - \sum_{j=1}^d w_j x_{ij} - w_0)^2$  


Для обучения нашей модели нам необходимо минизировать Q относительно W при заданных X,Y:  

$Q(w) = \frac{1}{N} \sum_{i=1}^N (y_i - \sum_{j=1}^d w_j x_{ij} - w_0)^2 \rightarrow \underset{W}{min}$ 

Оним из способов минимизации Q является градиентного спуск, алгоритм которого grad_descent($\alpha, \epsilon$, max_iter) в общем виде выглядит как:  
1. $W^{(0)} \leftarrow N(0,1)$ инициализировать начальное приближение для вектора весов небольшими случайными значениями, например из стандартного нормально распределения  
2. for i in range(max_iter):  
    2.1 $\nabla_wQ = [\frac{\partial Q}{\partial w_0}, \frac{\partial Q}{\partial w_1}, ..., \frac{\partial Q}{\partial w_d}]^T$ - посчитать градиент в точке $W = W^{(t)}$  
    2.2 $W^{(t+1)} = W^{(t)} - \alpha * \nabla_wQ$ - обновить вектор весов, сделав  шаг длины $\alpha$ в сторону антиградиента  
    2.3 Если $||\nabla_wQ ||_2 < \epsilon $  - если норма градиента стала меньше некоторой заданной малой величины, то мы считаем, что алгоритм сошелся и прерываем цикл
    
    
<b>Reading</b>:
1. http://www.machinelearning.ru/wiki/index.php?title=Simple_linear_regression
2. Bishop. Pattern Recognition and Machine Learning, 3.1
3. http://www.machinelearning.ru/wiki/index.php?title=Метод_градиентного_спуска

# Homework 5

Ваша задача - заполнить пропуски # YOUR CODE HERE, и выполнить код

В этой домашней работе необходимо реализовать одномерную линейную регрессию.  
Таким образом, в упрощенной постановке задачи d=1.

$f(x_i) = f(x_i; w) = w_1* x_i + w_0$   - наш алгоритм регрессии   
$Q(w) = \frac{1}{N} \sum_{i=1}^N (y_i - w_1 x_i - w_0)^2 \rightarrow \underset{w_1, w_0}{min}$ - наш MSE  

Градиент MSE по весам весам нашего алгоритма:  
$\nabla_wQ = [\frac{\partial Q}{\partial w_0}, \frac{\partial Q}{\partial w_1}]^T$

$\frac{\partial Q}{\partial w_0} = -\frac{2}{N} \sum_{i=1}^N (y_i - w_1 x_i - w_0)$  
$\frac{\partial Q}{\partial w_1} = -\frac{2}{N} \sum_{i=1}^N (y_i - w_1 x_i - w_0)x_i$  

Сам алгоритм градиентного спуска grad_descent($\alpha, \epsilon$, max_iter) будет выглядеть так:  

1. $w_1^{(0)}, w_0^{(0)} \leftarrow N(0,1)$ инициализировать начальное приближение для вектора весов небольшими случайными значениями, например из стандартного нормально распределения  
2. for i in range(max_iter):  
    2.1 $\nabla_wQ = [\frac{\partial Q}{\partial w_0}, \frac{\partial Q}{\partial w_1}]^T$ - посчитать градиент в точке $(w_1^{(t)}, w_0^{(t)})$  
    2.2 обновить вектор весов, сделав  шаг длины $\alpha$ в сторону антиградиента  
    $w_1^{(t+1)} = w_1^{(t)} - \alpha * \frac{\partial Q}{\partial w_1}$  
    $w_0^{(t+1)} = w_0^{(t)} - \alpha * \frac{\partial Q}{\partial w_0}$   
       
    2.3 Если $\sqrt{{\frac{\partial Q}{\partial w_0}}^2 + {\frac{\partial Q}{\partial w_1}}^2} < \epsilon $  - если норма градиента стала меньше некоторой заданной малой величины, то мы считаем, что алгоритм сошелся и прерываем цикл

In [1]:
import numpy as np

SEED = 17
np.random.seed(SEED)

In [10]:
class SimpleLinearRegression:
    def __init__(self, step = 0.01, tol = 1e-4, max_iter=1000, verbose=False, random_state=SEED):
        self.max_iter = max_iter # max iter count of gradient descent
        self.step = step # step of descent in the direction of antigradient
        self.tol = tol # we compare norm of gradient with that threshold
        self._w = None # w_1
        self._intercept = None # w_0
        self.random_state = random_state 
        self.verbose = verbose
        
    def predict(self, X):
        """
        estimate target variable "y" based on features X 
        """
        y_pred = self._w * X + self._intercept
        assert y_pred.shape[0] == X.shape[0]
        return y_pred
    
    def score(self, X, y):
        """
        MSE
        X - features
        y - true values of target variable
        """
        return np.mean((y - self.predict(X))**2)
    
    def _gradient(self, X, y):
        """
        Compute gradient of MSE subject to w_1, w_0
        X - features
        y - true values of target variable
        """
        grad_intercept = -2 * np.mean(y - self._w * X - self._intercept)
        grad_w = -2 * np.mean((y - self._w * X - self._intercept) * X)        
        return grad_w, grad_intercept
        
    def fit(self, X, y):
        """
        Train model with gradient descent
        X - features
        y - true values of target variable
        """
        # for reproducable results
        np.random.seed(self.random_state)
        
        # initialize weights
        self._w, self._intercept = np.random.randn(2)
        # perform gradient descent
        for iter in range(self.max_iter):
            # compute gradient at current W
            grad_w, grad_intercept = self._gradient(X, y)
            
            # make step, update W
            self._w -= self.step * grad_w
            self._intercept -= self.step * grad_intercept
            
            # compute gradient norm            
            grad_norm = np.sqrt(grad_w**2 + grad_intercept**2)
            
            # people like to watch how the error is reducing during iterations 
            if self.verbose:
                mse_score = self.score(X, y)
                print('iteration %d, MSE = %f, ||grad|| = %f' % (iter, mse_score, grad_norm))
                
            # compare gradient norm with threshold
            if grad_norm < self.tol:
                print('model converged')
                return self
        print('model did not converge')
        return self

### Прикладная задача
Для Boston dataset мы хотим научится предсказывать значение target по признаку CRIM.  
При необходимости посмотрите прошлый семенар и документацию к pandas.

In [3]:
import pandas as pd
from sklearn import datasets


In [4]:
boston_data = datasets.load_boston()
boston_data.keys()

dict_keys(['DESCR', 'feature_names', 'data', 'target'])

In [5]:
df = pd.DataFrame(boston_data['data'], columns=boston_data['feature_names'])
df['target'] = boston_data['target']
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
print(boston_data['DESCR'])

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [7]:
# реализуем функцию, которая считает MSE 
def mse_score(y_true, y_pred):
    """
    y_true - true values of target variable
    y_pred - predicted values of target variable 
    """
    result = np.mean((y_true - y_pred)**2)
    return result

Разделим наш датасет на 2 части: на первой части мы будем обучаться, на второй оценивать качество работы алгоритма на новых данных

In [8]:
from sklearn.model_selection import train_test_split

# разбили датасет в соотношении 60:40
df_train, df_test = train_test_split(df, test_size=0.4, random_state=SEED, shuffle=True)

In [11]:
# обучите модель на df_train c verbose=True
# Обратите внимание на отладочный вывод, ваша ошибка MSE должна уменьшаться с каждой итерацией
# мы хотим научится предсказывать значение target по признаку CRIM
model = SimpleLinearRegression(verbose=True)

X_train = df_train['CRIM']
y_train = df_train['target']

model.fit(X_train, y_train)

y_pred = model.predict(X_train)

mse_train_score = mse_score(y_train, y_pred)

print('MSE on train:', mse_train_score)

iteration 0, MSE = 620.150546, ||grad|| = 87.756208
iteration 1, MSE = 600.209317, ||grad|| = 50.506377
iteration 2, MSE = 582.999173, ||grad|| = 42.519621
iteration 3, MSE = 566.622634, ||grad|| = 40.753125
iteration 4, MSE = 550.808913, ||grad|| = 39.944626
iteration 5, MSE = 535.507063, ||grad|| = 39.278781
iteration 6, MSE = 520.696243, ||grad|| = 38.641496
iteration 7, MSE = 506.360119, ||grad|| = 38.016947
iteration 8, MSE = 492.483399, ||grad|| = 37.402820
iteration 9, MSE = 479.051351, ||grad|| = 36.798659
iteration 10, MSE = 466.049724, ||grad|| = 36.204263
iteration 11, MSE = 453.464727, ||grad|| = 35.619469
iteration 12, MSE = 441.283007, ||grad|| = 35.044121
iteration 13, MSE = 429.491642, ||grad|| = 34.478067
iteration 14, MSE = 418.078124, ||grad|| = 33.921156
iteration 15, MSE = 407.030344, ||grad|| = 33.373240
iteration 16, MSE = 396.336583, ||grad|| = 32.834174
iteration 17, MSE = 385.985497, ||grad|| = 32.303816
iteration 18, MSE = 375.966104, ||grad|| = 31.782025
ite

In [12]:
# # посчитали качество обученной модели на df_test

X_test = df_test['CRIM']
y_test = df_test['target']

y_pred = model.predict(X_test)
mse_test_score = mse_score(y_test, y_pred)

print('MSE on test:', mse_test_score)

MSE on test: 74.28804282955323
